# Assignment 3

### Question 1: Policy iteration

Consider the problem of Colonel Kurtz (the Jupyter notebook and .m files are on Blackboard). See Chapter 5 of Stachurski (2009) for a more rigorous exposition of the problem.

**a) We discussed that there are three methods for performing the policy evaluation step of the policy iteration algorithm.  Formalize each of these methods mathematically in the setting of Colonel Kurtz problem, using proper matrix notation when appropriate.**





%Matrix Inversion

$v_{x}^*= \max_{[a \in \Gamma(x)]}[u(x-a)+ \rho \sum\limits_{z=0}^\beta P_{ij}^x v_{j}(a+z) \phi (z)]$


$P_{ij}^x$  is the transition matrix 

$Iv_{x}* - \rho \sum\limits_{z=0}^\beta P_{ij}^x v_{j}(a+z)= u^{\alpha n}$

$( I + \rho \Pi^{\alpha n} + (\rho \Pi^{\alpha n})^2 +  ... )$$u^{\alpha n} $

$v_{x}^*=v_{x}$


%Successive Aproximation 

$v_{n+1}(i)=\max_{[a \in \Gamma(x)]}[u(x-a)+ \rho \sum\limits_{z=0}^\beta P_{ij}^x v_{j}(a+z) \phi (z)]$
and let $[f_{n+1} \in arg \max_{[a \in \Gamma(x)]}[u(x-a)+ \rho \sum\limits_{z=0}^\beta P_{ij}^x v_{j}(a+z) \phi (z)]$

let $\mu_{n}= sup_{i} [v_{n(i)} - v_{n-1 (i)}] 
stop if \frac{c\gamma\mu_{n}}{1-\gamma} < \epsilon$

Theorem 

$\mu_{n} = sup_{i} |v_{n} (i) - v_{n-1} (i)| \leq c\gamma^{n-1}sup|v_{i}+ v_{0}(i)$
Let further $\frac{c\gamma\mu_{n}}{(1-\gamma)} < \epsilon$

$ sup_{i}|v_{n}(i) - v*(i)|\leq \epsilon $
and 

$v_{n}(i) - \mu_{n} \frac{c\gamma}{1-\gamma} \leq v^{fn} (i) \leq v*^{fn}(i)+v_{n}(i) + \mu_{n}\frac{c\gamma}{1-\gamma}$

So that $0\leq sup_{i} (v*(i) - v^{fn}(i))\leq 2\epsilon$

The sequence ${f_n}$ contains at leats one limit point, each limit poing defines an optimal stationary, deterministic strategy 


%Truncated Matrix

$A^{(k)} =\sum\limits_{z=0}^\beta P_{ij}^x v_{j}(a+z)\phi(z)$ is truncated by $1 \leq \beta \leq \infty$

$v_{n+1}= u^{n} + A^{k(n)} {(v^\alpha-v)}$


=$u^{n} +{\alpha P(f_{n})}{r(f_{n}+ ...+{\alpha P(f_{n})}^{\beta(n)-1}r(f_{n}+{\alpha P(f_{n})}^\beta(n) (v^\alpha)}$





 






**b) Which of these three methods is used in the code? Write the code for the other two methods. Show that these produce the same results.**


In [ ]:
%Matrix Inversion
function v_alpha = policyEval2(alpha)
    global rho S

    % Set up \Pi^{\alpha} :
    len = length(S);
    Pi_alpha = zeros(len,len);
    y = S;
    for x = S
        Pi_alpha(x+1,:) = phi(y - alpha(x+1));
    end
    

    
    r_alpha = zeros(len:1);
for x = S
    c = max (0, x-alpha(x+1));
    r_alpha(x+1) = flowUtility(c);
    
end

%inverting matrix

v_alpha = (inv(eye(size(Pi_alpha))-rho.*Pi_alpha).*r_alpha)'

    


end







In [ ]:
%Successive Aproximation 
function v_alpha = policyEval2(alpha)
    global rho S

    % Set up \Pi^{\alpha} :
    len = length(S);
    Pi_alpha = zeros(len,len);
    y = S;
    for x = S
        Pi_alpha(x+1,:) = phi(y - alpha(x+1));
    end
    
    r_alpha = zeros(len:1);
for x = S
    c = max (0, x-alpha(x+1));
    r_alpha(x+1) = flowUtility(c);
    
policy = @(v)(bellman(v)>0);
NPi = 20;

tolPolicyValue = 1E-9;

% Solve by policy function iteration
v=0; vNew=1; dist_polSA = [];
while max(abs(v-vOld))>1E-9
    v = vNew;
    alpha = policy(v);
    % The loop for finding the fixed point for policy alpha
    vOut = v; 
    vIn = v+1; % This is to make sure the loop will start
    while max(abs(vIn-vOut))>tolPolicyValue
        vIn = vOut;
        vOut = (u+rho*(Pi_alpha*vIn)')'.*alpha;
    end
    
    v = vOut;
    dist_polSA = [dist_polSA;max(abs(vNew-v))];
end

**c) Compare between the run time of value function iteration and policy iteration, separately using each of the three methods for computing the policy evaluation step. What are the settings that will affect the results of this comparison? Summarize these effects and provide intuition for your results.**

Value iteration is faster per cycle but requieres many cycles. 
On the other hand, Policy iteration is slower per cycle but 
requieres few cycles. 


### Question 2: Bayesian learning

Consider a simplified (but infinite horizon) version of the Bayesian learning problem in Chan and Hamilton (2006). An agent is initially engaged in an activity that gives utility $u(Y_t)$, with $u$ increasing, from the activity's payoff $$Y_t=\theta+\epsilon_t$$ 

in period $t$. In each period $t$, the agent chooses to either continue the activity ($A_t=1$) or permanently discontinue it ($A_t=0$) and collect an outside value of zero. The payoff $Y_t$ is not known when the agent makes this decision, but only arrives right after if the agent chooses to stay. The agent only observes $Y_t$ and not its components, but she does know that $\theta\sim N(\mu,\sigma^2)$ and $\epsilon_t\sim N(0,\eta^2)$ independently from $\theta$ and over time. She maximizes expected utility discounted with factor $\rho$.

*Note:  This simplified learning problem is at the heart of Jovanovic (1982) on firm learning and survival. Abbring and Campbell (2005) used a slight extension to learning about an autoregressive process, including some possibly useful (for the questions below) discussion of the dynamics of this type of learning.*

**a) For an agent active up to time $t$ ($A_{t-1}=1$), the entire payoff history $(Y_1,\ldots,Y_{t-1})$ is informative on $\theta$ and therefore on payoffs in period $t$ and beyond. However, under the stated assumptions, the posterior mean $\hat{\theta}_{t}=\mathbb{E}[\theta|Y_1,\ldots,Y_{t-1}]$ and variance $\Sigma_{t}=var(\theta|Y_1,\ldots,Y_{t-1})$ suffice. Explain.**

It suffices because the prior variable  is normal, meaning that the post  one will be normal as well. To prove that please consider $Y_t=\theta+\epsilon_t$ where $\theta\sim N(\mu,\sigma^2)$ and $\epsilon_t\sim N(0,\eta^2)$. While $Y_t$ is observed, $\theta$ it is not.    
$\theta\sim N(\mu,\sigma^2)$ is known and therefore we can derive that $Y_t$ $\sim N(\mu,\sigma^2+ \eta^2)$. Conditional on observing the outcome of $\theta$ we can instead write:   
$Y_t$| $\theta\sim N(\mu,\eta^2)$, since know the only source of uncertainty is the error term $\epsilon_t$. This implies that $f(Y_1,.....Y_{t-1}\theta|)$ will equal the product of the independent normally distributed pdfs. 
 Thus,the above expression can be written such as:   
$f(Y_1,.....Y_{t-1}\theta|)$ $\propto$ $\displaystyle\prod_{j=1}^{t-1}$ [$\exp {-\frac{(Y_t-\theta)^2}{2\eta^2}] exp[ - \frac{(\theta-\mu)^2}{2\sigma^2}}] $ $\propto$  $\exp {-\frac{\sum_{j=1}^{t-1}(Y_j-\theta)^2}{2\eta^2} - \frac{(\theta-\mu)^2}{2\sigma^2}} $. $\propto$ [$\exp {\frac{\sum_{j=1}^{t-1}(Y_j^2+ \theta^2 -2\theta Y_j)}{2\eta^2} - \frac{(\theta^2+\mu^2-2\mu\theta)}{2\sigma^2}} $] $\propto$ $\exp [{\frac{-\sigma^2\sum_{j=1}^{t-1}(Y_j^2+ \theta^2 -2\theta Y_j)- \eta^2(\theta^2+\mu^2-2\theta\mu)}{2\eta^2\sigma^2}} $] $\propto$ $\exp [{\frac{-\sigma^2\sum_{j=1}^{t-1}(Y_j^2)+2\sigma^2+ \sum_{j=1}^{t-1}(Y_j)-\sigma^2[(t-1)\theta^2] -\eta^2\theta^2 - \eta^2\mu^2+2\eta^2\theta\mu)}{2\eta^2\sigma^2}} $]  
$\propto$ $\exp [{\frac{-\theta^2[\sigma^2(t-1)+\eta^2]+2\theta[\eta^2\mu+\sigma^2 \sum_{j=1}^{t-1}(Y_j)]-\eta^2\mu^2-\sigma^2\sum_{j=1}^{t-1}(Y_j)^2)}{2\eta^2\sigma^2}} $]   

$\propto \exp [\frac{A}{V}]$ $\propto \exp [\frac{B}{V}]$ $-\exp [\frac{C}{Z}]$, where:  

A= - $\theta^2 + \frac{2\theta[\eta^2\mu+\sigma^2\sum_{j=1}^{t-1}(Y_j)}{\sigma^2(t-1)+\eta^2}- \frac{\eta^2\mu^2+\sigma^2 \sum_{j=1}^{t-1}(Y_j^2)}{\sigma^2(t-1)+\eta^2}$

B= - $\theta^2 + \frac{2\theta[\eta^2\mu+\sigma^2\sum_{j=1}^{t-1}(Y_j)}{\sigma^2(t-1)+\eta^2}-[\frac{2\theta(\eta^2\mu^2+\sigma^2 \sum_{j=1}^{t-1}(Y_j^2]}{\sigma^2(t-1)+\eta^2}$

C= $ \eta^2 \mu^2+\sigma^2 \sum_{j=1}^{t-1}Y_j^2$

Z= $\sigma^2(t-1)+\eta^2$

V= $ \frac {2\eta^2\sigma^2}{\sigma^2(t-1)+\eta^2}$  

Where the last result has been obtained via multiplying by the constant term $ \exp [- \frac{2\theta[\eta^2\mu+\sigma^2 \sum_{j=1}^{t-1}(Y_j]}{\sigma^2(t-1)+\eta^2}]^2 $ which does not depend on $\theta$. 

Lastly, $f(Y_1,.....Y_{t-1}\theta|)$ $\propto$ $ \exp(-\frac{J}{K})$ = $\exp (-\frac{(\theta- \hat{\theta})^2}{2\Sigma_t}) $, where:  
J= [$\theta- \frac{[\eta^2\mu+\sigma^2 \sum_{j=1}^{t-1}(Y_j)]}{\sigma^2(t-1)+\eta^2}]^2 $  
K= $\frac{2\eta^2\sigma^2}{\sigma^2(t-1)+\eta^2}$  
Finally it results that=> $f(\theta|Y_1,.....Y_{t-1})$ $c\sim N(\hat{\theta_t}, \Sigma_t)$, where

$\hat{\theta}_{2}=\frac{\eta^2\mu + \sigma^2  \sum_{j=1}^{t-1}(Y_j)]}{\sigma^2 (t-1)+ \eta^2}$ 
${\Sigma}_{2}= \frac{\eta^2\sigma^2}{\sigma^2 (t-1)+\eta^2}$

**b) Suppose that the agent has a rational prior: $\hat\theta_1=\mu$ and $\Sigma_1=\sigma^2$. Show that, for $t\geq 2$, $$\hat{\theta}_t=\hat{\theta}_{t-1}+\lambda_t\left(Y_{t-1}-\hat{\theta}_{t-1}\right)\text{ and }\Sigma_t=(1-\lambda_t)\Sigma_{t-1},$$ with $\lambda_t=\Sigma_{t-1}/(\Sigma_{t-1}+\eta^2)$. Note that $\lambda_t=1$ if the agent observes $\theta$ without noise ($Y_t=\theta$) and $\lambda_t\downarrow 0$ if the observations become uninformative about $\theta$ ($\eta^2\rightarrow\infty$ or $\Sigma_{t-1}\downarrow 0$).**

The problem can be solved via using the Kalman filter as in the work of Abbring and Campbell (2005).  However for now we are going to use proof by induction. 

we know already what is $\hat{\theta}_{1}$ and ${\Sigma}_{1}$. So for $\hat{\theta}_{2}$ and ${\Sigma}_{2}$, we replace:

$\hat{\theta}_{2}=\frac{\eta^2\mu + \sigma^2\sum_{j=1}^{t-1} y_{j}} {\sigma^2 + \eta^2}$ 
$\hat{\theta}_{2}=\frac{\eta^2\mu + \sigma^2 y_{1}}{\sigma^2 + \eta^2}$ 

${\Sigma}_{2}= \frac{\eta^2\sigma^2}{\sigma^2_{t-1}+\eta^2}$
${\Sigma}_{2}= \frac{\eta^2\sigma^2}{\sigma^2_{2}+\eta^2}$

Now we take the equation 

$\hat{\theta}_t=\hat{\theta}_{t-1}+\lambda_t(Y_{t-1}-\hat{\theta}_{t-1})$

$\hat{\theta}_2=\hat{\theta}_{1}+\frac{\Sigma_{1}}{\Sigma_{1}+\eta^2}(Y_{1}-\hat{\theta}_{1})$

$\hat{\theta}_2=\mu + \frac{\sigma^2}{\sigma^2+\eta^2} (y_{1} -\mu) $

$\hat{\theta}_2=\mu + \frac{y_{1}\sigma^2 -\mu\sigma^2}{\sigma^2+\eta^2}   $

$\hat{\theta}_2=\frac{\mu\sigma^2+ \eta^2\mu  + y_{1}\sigma^2 -\mu\sigma^2}{\sigma^2+\eta^2}   $

$\hat{\theta}_2=\frac{\eta^2\mu  + y_{1}\sigma^2 }{\sigma^2+\eta^2}   $


$\Sigma_t=(1-\lambda_t)\Sigma_{t-1}$

$\Sigma_2=(1-\frac{\Sigma_{t-1}}{\Sigma_{t-1}+\eta^2})\Sigma_{t-1}$

$\Sigma_2=(1-\frac{\Sigma_{1}}{\Sigma_{1}+\eta^2})\Sigma_{1}$

$\Sigma_2=(1-\frac{\Sigma_{1}}{\Sigma_{1}+\eta^2})\Sigma_{1}$

$\Sigma_2=(1-\frac{\sigma^2}{\sigma^2+\eta^2})\sigma^2$

$\Sigma_2=\frac{\sigma^2(\sigma^2+\eta^2)-\sigma^2\sigma^2}{\sigma^2+\eta^2}$

$\Sigma_2=\frac{\sigma^2\eta^2}{\sigma^2+\eta^2}$

Since it is true for t = 2 by induction, we will assume that it is true for t-1. and demonstrate that it is true for t 

What happen to $\hat{\theta}_t$ 

$\hat{\theta}_t=\hat{\theta}_{t-1}+\lambda_t(Y_{t-1}-\hat{\theta}_{t-1})$

$\hat{\theta}_t=\hat{\theta}_{t-1}+\frac{\Sigma_{t-1}}{\Sigma_{t-1}+\eta^2}(Y_{t-1}-\hat{\theta}_{t-1})$

$\hat{\theta}_t=\hat{\theta}_{t-1}+\frac{Y_{t-1}\Sigma_{t-1}}{\Sigma_{t-1}+\eta^2}-\frac{\hat{\theta}_{2}\Sigma_{t-1}}{\Sigma_{t-1}+\eta^2}$

After we replace $\Sigma_{t-1}=\frac{\sigma^2\eta^2}{\sigma^2+\eta^2})$ and $\hat{\theta}_{t-1}=\frac{\eta^2\mu  + y_{t-1}\sigma^2 }{\sigma^2+\eta^2}   $ , we get 

$\hat{\theta}_t=\frac{\eta^2\mu + \sigma^{2}\sum_{j=1}^{t-1} y_{j} }{\sigma^2(t-1)+\eta^2}$

If we add and substract $\hat{\theta}_{t-1}$ 

$\hat{\theta}_t=\frac{\eta^2\mu + \sigma^{2}\sum_{j=1}^{t-1} y_{j} }{ \sigma^2(t-1)+\eta^2} - \frac{\eta^2\mu  + \sum_{j=1}^{t-2} y_{j}\sigma^2 }{\sigma^2(t-1)+\eta^2}+ {\theta}_{t-1} $

$\hat{\theta}_t=\frac{(\eta^2\mu + \sigma^{2}\sum_{j=1}^{t-1} y_{j}) ( \sigma^{2}(t-2)+\eta^2)-(\eta^2\mu+\sigma^2 \sum_{j=1}^{t-2})}{ \sigma^2(\sigma^2(t-2)+\eta^2)+(\sigma^2(t-2)+\eta^2)^2} + {\theta}_{t-1} $

We simplified $\sigma^2(t-2)+\eta^2$

$\hat{\theta}_t=\frac{\sigma^2 y_{t} - \sigma^2 {\theta}_{t-1}}{(\sigma^2(t-1)+\eta^2}+ {\theta}_{t-1}$

$\hat{\theta}_t=\frac{\sigma^2 y_{t}}{(\sigma^2(t-1)+\eta^2} * (y_{t} -{\theta}_{t-1}) + {\theta}_{t-1}$

we get 

$\hat{\theta}_t=\hat{\theta}_{t-1}+\lambda_t(Y_{t-1}-\hat{\theta}_{t-1})$

What happen to $\Sigma_t$?

$\Sigma_t=(1-\lambda_t)\Sigma_{t-1}$

$\lambda_t=\frac{\Sigma_{t-1}}{(\Sigma_{t-1}+\eta^2)}$

$\Sigma_t=\frac{\eta^2\sigma^2}{\sigma^2(t-1)+\eta^2}*\Sigma_{t-1}*\frac{1}{\Sigma_{t-1}}$

$\Sigma_t=\frac{\eta^2\sigma^2}{\sigma^2(t-1)+\eta^2}* \frac{\sigma^2(t-2)+\eta^2}{\eta^2\sigma^2}*\Sigma_{t-1}$

$\Sigma_t=\frac{\sigma^2(t-2)+\eta^2}{\sigma^2(t-1)+\eta^2}*\Sigma_{t-1}$

so finally we get 

$\Sigma_t=(1-\lambda_t)\Sigma_{t-1}$

It proves that for $t\geq 2$, $$\hat{\theta}_t=\hat{\theta}_{t-1}+\lambda_t\left(Y_{t-1}-\hat{\theta}_{t-1}\right)\text{ and }\Sigma_t=(1-\lambda_t)\Sigma_{t-1},$$ with $\lambda_t=\Sigma_{t-1}/(\Sigma_{t-1}+\eta^2)$




**c) Note that $\Sigma_t$ is a deterministic function of the parameters and time, so that, for given parameters, we can represent the state when the agent decides in period $t$ as $(\hat\theta_t,t)$. Write the Bellman equation for the expected discounted utility $v(x,t)$ in state $(\hat\theta_t,t)=(x,t)$. Please also specify the conditional distribution of $x',t+1|x,t$, where $x'$ is next-period state.**

This equation comes from the assumption that the distribution of v  is independent of the threatment. Hence we have the following expression

Bellman equation:  
v(x,t)= $ max {E[u(Y_t)+ \rho v(x', t+1)|x,t],0}  $  

$ v(\theta=x)=max{E[u(Y_t)|\theta=x]+\rho v(\theta=x),0} $

The distribution of (x', t+1)|x,t is given by:   
x',t+1|x,t $\sim N(E[\hat\theta_{t+1}|[\hat\theta_{t},t], V [[\hat\theta_{t+1}|[\hat\theta_{t}, t]) $  where   

$ E[\hat\theta_{t+1}|[\hat\theta_{t},t]= E[\hat\theta_{t}+\lambda_{t+1}(Y_t- \hat\theta_{t})|\hat\theta_{t},t]=\hat\theta_{t}+\lambda_{t+1}(E[Y_t|\hat\theta_{t},t]-\hat\theta_{t})= \hat\theta_{t}+ \lambda_{t+1}(E[\epsilon|t] + E[\theta|\hat\theta_{t},t]-\hat\theta_{t})=\hat\theta_{t}+\lambda_{t+1}(\hat\theta_{t}-\theta_{t})=\hat\theta_{t} $   
and  $ V [\hat\theta_{t+1}|[\hat\theta_{t}, t]) = V( \hat\theta_{t} + \lambda_{t+1}(Y_t-\hat\theta_{t})|\hat\theta_{t},t]= V[\lambda_t+ 1Y_t|\hat\theta_{t}, t]=\lambda^2_{t+1} V[\hat\theta+ \epsilon_t|\hat\theta_{t},t] $= ..........
....= $\lambda^2_{t+1}(V[\hat\theta|\hat\theta_{t},t]+V[\epsilon|t]+2Cov[\epsilon_t, \hat\theta|\hat\theta_{t},t])= \lambda^2_{t+1}(\Sigma_t+\eta^2) $


**d) Show that the value $v(x,t)$ increases with $x$. What does this imply for the optimal decision rule?**

Following the reasoning of Jovanovic (1982) we have:  
Assume that S  is a contraction operator with modulus $\rho$ < 1  
Let S denote the operator in the above Bellman Equation so that V is the unique solution to the equation
 V = SV.     
 Since S is a contraction operator, we have V = lim$_{n-> \infty}$S$^ng$ where g is any bounded, continuous
 function (where $Sn^g$ =S$(S^{n-1}g$)). Thus, any  monotonicity property of one of the members of the sequence
 S$^ng$ is also valid for (weakly)  the limit function V.  
 Thus if a function g is increasing in x, than so it will be Sg. But V is a function of $u(Y_t)$ and it is known that  $u$ is increasing.  
 By induction, this propertiy carries over to the limit function V which is ultimately increasing in x given the positive value of this. 
 Concerning the optimal decision rule, if v always increase, it is more difficult to find and optimal value; however, there exists a treshold value $\alpha_t$ at which the agent chooses to exit if $Y_t$≤$\alpha_t$. In this order of ideas, even v increases as x is greater, the treshold allow us to find an optimal value. 
 

**e) Note that the problem is a nonstationary, infinite horizon problem. However, what happens to the posterior variance $\Sigma_t$ when $t$ grows large? What does this suggest about $v(x,t)$ and optimal decisions at large $t$? How could you use this to compute $v(x,t)$ at earlier $t$?**

Given that the posterior variance depends on t on the denominator, we would expect that as t increases, the posterior variance decrease reaching almost value of zero. At large t, the optimal decision will be independent of $\hat\theta_t$ so it will be equal to $\theta_t$. In this case, we can rewrite the Bellman equation as $ v(\theta=x)=max{E[u(Y_t)|\theta=x]+\rho v(\theta=x),0} $. 

To compute this at earlier t, we can simulate a long time peiod of time (normally t=10000 for research cases), and v(\theta=x,t) will be the fixed point. Therefore, we can solve it using backward recursion. 

**f) In doing so, to evaluate $v(x,t)$ by iterating on the Bellman equation, you will have to compute expectations over $Y_t$ and $\hat\theta_{t+1}$ given $\hat\theta_t=x$ (and $\Sigma_t$). A new problem you will encounter here is that $Y_t$ and $\hat\theta_t$ can take values on all of $\mathbb{R}$. Discuss possible ways to deal with this.**

The computational technique precomputation of integrals makes possible to construct conditional expectation functions 
in dynamic stochastic models in the initial stage of a solution procedure. This technique is very general: it works for 
a broad class of approximating functions, and it is compatible with both continuous-state and discrete-state shocks. 
To avoid to encounter values on all  $\mathbb{R}$, we could use an interval of the possible expected values of $y_{t}$

### References

- Abbring, Jaap and Jeffrey Campbell (2005). A firm's first year. Discussion Paper 2005-046/3. Tinbergen Institute, Amsterdam. http://dare.ubvu.vu.nl/bitstream/handle/1871/9660/05046.pdf
- Chan, Tat and Barton Hamilton (2006). Learning, private information, and the economic evaluation of randomized experiments. *Journal of Political Economy* 114, 997-1040. http://www.jstor.org/stable/10.1086/508239
- Jovanovic, Boyan (1982). Selection and the Evolution of Industry. *Econometrica* 50, 649-670. http://www.jstor.org/stable/1912606
- Stachurski, John (2009). Economic dynamics: theory and computation. MIT Press.
